In [1]:
import pandas as pd
import numpy as np
from scipy.io import savemat
import glob
import os

In [2]:
folder_path = './9_Strains/setA'  # Replace with your folder path

txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

combined_df = pd.DataFrame()
grp = []
column_index = 0
for file in txt_files:
    try:
        df = pd.read_csv(file, sep='\t', header=None)  
        base_filename = os.path.splitext(os.path.basename(file))[0]
        rename = base_filename[:-2]
        # df.columns = ['x'] + [f'Spectrum{i}' for i in range(1, df.shape[1])]
        df.columns = ['Wavenumber'] + [f'Spectrum{column_index + i}' for i in range(1, df.shape[1])]
        column_index += df.shape[1] - 1  # Update index for next file
        for _ in range(1, df.shape[1]):
            grp.append(rename)
        if combined_df.empty:
            combined_df = df
            print("jhol")
        else:
            combined_df = pd.merge(combined_df, df, on='Wavenumber', how='outer')
    except Exception as e:
        print(f"Error processing file {file}: {e}")

combined_df.fillna(combined_df.median(), inplace=True)

print(combined_df.head())  # To check the top few rows

os.makedirs('./csv_data', exist_ok=True)
combined_df.to_csv('./csv_data/known_spectra.csv', index=False)


jhol
   Wavenumber  Spectrum1  Spectrum2  Spectrum3  Spectrum4  Spectrum5  \
0     542.245      462.8      463.8      464.6      463.0      466.8   
1     542.328      464.4      460.6      462.4      464.2      468.2   
2     542.411      464.0      462.6      462.4      465.0      467.6   
3     542.493      462.6      461.0      460.4      463.0      463.8   
4     542.576      461.8      461.0      461.4      467.4      465.8   

   Spectrum6  Spectrum7  Spectrum8  Spectrum9  ...  Spectrum639  Spectrum640  \
0      464.2      467.8      466.8      466.0  ...        486.6        536.0   
1      467.8      464.8      462.8      467.8  ...        489.4        533.8   
2      466.2      465.4      464.8      471.4  ...        484.8        536.4   
3      466.2      465.2      466.0      469.8  ...        482.8        531.2   
4      466.4      467.2      467.0      469.2  ...        482.8        535.8   

   Spectrum641  Spectrum642  Spectrum643  Spectrum644  Spectrum645  \
0        55

In [3]:
grp_pd = pd.DataFrame(grp, columns=['Labels'])
grp_pd.to_csv("./csv_data/labels.csv", index=False)

In [9]:
folder_path = './9_Strains/setB'  # Replace with your folder path

txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

combined_df = pd.DataFrame()
grp = []
column_index = 1
for file in txt_files:
    try:
        df = pd.read_csv(file, sep='\t', header=None)  
        base_filename = os.path.splitext(os.path.basename(file))[0]
        rename = base_filename[:-2]
        # df.columns = ['x'] + [f'Spectrum{i}' for i in range(1, df.shape[1])]
        df.columns = ['Wavenumber'] + [f'Unknown{column_index + i}' for i in range(1, df.shape[1])]
        column_index += df.shape[1] - 1  # Update index for next file
        for _ in range(1, df.shape[1]):
            grp.append(rename)
        if combined_df.empty:
            combined_df = df
            print("zhol")
        else:
            combined_df = pd.merge(combined_df, df, on='Wavenumber', how='outer')
    except Exception as e:
        print(f"Error processing file {file}: {e}")

combined_df.fillna(combined_df.median(), inplace=True)

print(combined_df.head())  # To check the top few rows

os.makedirs('./csv_data', exist_ok=True)
combined_df.to_csv('./csv_data/unknown_spectra.csv', index=False)


zhol
   Wavenumber  Unknown2  Unknown3  Unknown4  Unknown5  Unknown6  Unknown7  \
0     542.245     456.0     454.6     454.8     478.8     472.6     465.6   
1     542.328     458.2     454.0     452.2     474.2     472.2     464.0   
2     542.411     459.4     453.8     457.6     481.0     474.2     466.0   
3     542.493     456.4     456.0     452.6     477.2     475.6     465.6   
4     542.576     455.0     455.0     451.6     473.0     466.8     462.6   

   Unknown8  Unknown9  Unknown10  ...  Unknown285  Unknown286  Unknown287  \
0     476.2     474.8      469.6  ...       505.8       509.6       552.0   
1     474.6     476.8      468.6  ...       497.6       509.6       550.2   
2     478.4     474.4      470.0  ...       501.8       513.6       546.6   
3     478.0     471.8      468.8  ...       502.8       507.4       547.4   
4     476.8     474.0      468.8  ...       502.0       508.6       547.8   

   Unknown288  Unknown289  Unknown290  Unknown291  Unknown292  Unknow

In [4]:


# Load data from CSV files
known_spectra_df = pd.read_csv('./csv_data/known_spectra.csv')
unknown_spectra_df = pd.read_csv('./csv_data/unknown_spectra.csv')
labels_df = pd.read_csv('./csv_data/labels.csv')
groups_df = pd.read_csv('./csv_data/grp.csv')

# Extract data from DataFrames
Akn = known_spectra_df.drop(columns=['Wavenumber']).values
xorg = known_spectra_df['Wavenumber'].values
Au = unknown_spectra_df.drop(columns=['Wavenumber']).values
labels = labels_df['Labels'].values
grp = groups_df['Group'].values

# Prepare dictionary to save in .mat file
mat_dict = {
    'Akn': Akn,
    'grp': grp,
    'xorg': xorg,
    'Au': Au,
    'labels': labels
}

# Save dictionary to .mat file
savemat('./csv_data/combined_spectra.mat', mat_dict)
